###**La pyramide :**
chaque joueur à n cartes, et il y a des cartes au centre (en forme de pyramide).

A chaque tours une carte du centre est retournée, et si un joueur a la carte il donne le nombre de point conséquent à son adversaire.

Il peut bluffer, et si il se fait démasquer, il en prend le double. Si le joueur qui a accusé à tord, il prend 3 points en plus.

Si les deux claim la carte, il y en a un des deux choisi aleatoirement qui peut accuser l'autre de mentir.

Le but de la partie est d'avoir le moins de points.

###**Le bluff :**
En fonction des cartes dans la main, on a plus de chance d'accuser l'autre si il accuse sur des cartes qu'on a en main.

Quand le joueur accusé de mensonge **ment, il n'a *pas* a montrer sa carte**, si il dit la verité il doit montrer la carte concerné.

###**IA :**
Au fur et a mesure des manche, si l'IA voit que le joueur ment souvent, elle aura + de chance d'accuser l'autre joueur.

L'IA peut retenir les cartes de l'autre. SI elle a toutes les cartes du joueur en mémoire, elle l'accuse systematiquement si il ment. Si elle sait qu'il a la carte qu'il claim, elle ne l'accuse pas.

###**Limites :**
- Si le joueur naif nous accuse tout le temps, l'ia va moins bluffer mais surtout elle aura tendance à + rater ses bluff si elle se fait accuser tout le temps.
- On peut alors changer les points attribués en fonction de si on a un bluff raté, une mauvaise denonciation etc mais on devra alors modifier les probas de l'ia pour qu'elle s'adapte, il vaut mieux laisser l'atribution de points fixe et essayer de renforcer l'ia pour qu'elle se defende quand elle se fait trop accuser.
- On estime a 0,3 la probabilité de l'adversaire a partir de laquelle notre IA va ne jamais bluffer afin de ne pas se penaliser. Plus la proba de l'adversaire sera haute a partir de la, plus il aura de chance de perdre car dans ce jeu, abuser de la strategie d'accusation permet grandement d'augmenter ses chances de gagner, il faut donc une strategie pour contrer cela.


In [1]:
import random

class Carte:
    def __init__(self, valeur):
        self.valeur = valeur

    def __repr__(self):
        return str(self.valeur)

class Deck:
    def __init__(self):
        self.cartes = [Carte(valeur) for valeur in range(1, 100) for _ in range(4)]  # 4 cartes par valeur
        random.shuffle(self.cartes)

    def tirer_carte(self):
        return self.cartes.pop() if self.cartes else None

class Main:
    def __init__(self, cartes):
        self.cartes = cartes

    def possede_carte(self, valeur):
        return any(carte.valeur == valeur for carte in self.cartes)

class Joueur:
    def __init__(self, nom, probaBluff=0.2, probaDenonce=0.6):
        self.nom = nom
        self.main = None
        self.points = 0
        self.probaBluff = probaBluff  # Probabilité de bluff du joueur
        self.probaDenonce = probaDenonce  # Probabilité de dénonciation du joueur

    def recevoir_main(self, cartes):
        self.main = Main(cartes)

    def ajouter_points(self, points):
        self.points += points

    def claim_carte(self, valeur):
        # Le joueur bluffe s'il a une probabilité de bluff > une valeur aléatoire ou s'il possède la carte
        return random.random() < self.probaBluff or self.main.possede_carte(valeur)

    def accuser(self, valeur):
        # Le joueur accuse s'il pense que l'autre bluffe
        return random.random() < self.probaDenonce

class AdversaireIA(Joueur):
    def __init__(self, nom="IA", probaBluff=0.2, probaDenonce=0.2):
        super().__init__(nom, probaBluff, probaDenonce)
        self.cartesAdversaire = []  # Mémorise les cartes de l'adversaire
        self.bluffReussis = 0
        self.bluffRate = 0
        self.denonceReussi = 0
        self.denonceRate = 0
        self.cptAccuse = 0
        self.stopMentir = False

    def recalcul_proba(self, adversaire,numManche):
      # Si l'adversaire dénonce souvent (>= 0.3), l'IA cesse presque totalement de bluffer
      #print('cpt', self.cptAccuse)
      #print('manche', numManche)
      #print(self.cptAccuse/numManche)
      '''if (self.cptAccuse >= 1 and numManche == 20 ):
        self.stopMentir = True
        self.probaBluff = 0.01
        print("true")
      elif(self.stopMentir):'''
      if adversaire.probaDenonce >= 0.3:
        self.probaBluff = 0.01
      else:
        self.probaBluff = max(0.01, self.probaBluff + (self.bluffReussis - self.bluffRate) * 0.05)

      self.probaDenonce = min(0.5, 0.2 + (self.denonceReussi - self.denonceRate) * 0.05)

    def accuser(self, valeur):
        # Si la carte est connue comme appartenant au joueur, ne pas accuser
        if valeur in self.cartesAdversaire:
            return False
        # Si l'IA connaît toutes les cartes du joueur, elle accuse systématiquement un bluff
        elif len(self.cartesAdversaire) == len(self.main.cartes):
            return True
        # Sinon, comportement normal avec probabilité
        return random.random() < self.probaDenonce

def distribuer(deck, joueur1, ia, nb_cartes_main=4):
    joueur1.recevoir_main([deck.tirer_carte() for _ in range(nb_cartes_main)])
    ia.recevoir_main([deck.tirer_carte() for _ in range(nb_cartes_main)])

def manche(carte_retournee, joueur1, ia, numeroManche):
    # Liste des joueurs qui prétendent posséder la carte retournée (soit l'IA, soit le joueur humain)
    joueurs_claims = [joueur for joueur in [joueur1, ia] if joueur.claim_carte(carte_retournee.valeur)]

    # Si personne ne revendique la carte, la manche s'arrête ici
    if not joueurs_claims:
        return

    # Sélection aléatoire d'un joueur parmi ceux qui ont revendiqué la carte
    joueur_claim = random.choice(joueurs_claims)

    # Détermine qui est l'adversaire (le joueur opposé à celui qui a revendiqué la carte)
    adversaire = joueur1 if joueur_claim == ia else ia

    # L'adversaire décide s'il accuse le joueur qui a revendiqué la carte d'un bluff
    accuse = adversaire.accuser(carte_retournee.valeur)

    if accuse:  # Si l'adversaire accuse un bluff
        if isinstance(joueur_claim, AdversaireIA):
                joueur_claim.cptAccuse += 1
        if joueur_claim.main.possede_carte(carte_retournee.valeur):
            #  L'accusation était FAUSSE → Le joueur revendiquait la carte légitimement
            adversaire.ajouter_points(3)  #  L'adversaire est puni fortement pour une accusation erronée (3 points )

            if isinstance(adversaire, AdversaireIA):
                adversaire.denonceRate += 1  # L'IA enregistre une dénonciation ratée
                adversaire.cartesAdversaire.append(carte_retournee.valeur)  # L'IA mémorise la carte correcte du joueur
        else:
            #  L'accusation était JUSTE → Le joueur revendiquait une carte qu'il n'avait pas (bluff)
            joueur_claim.ajouter_points(2)  #  Le bluffeur est puni et l'accusateur gagne 2 points

            if isinstance(joueur_claim, AdversaireIA):
                joueur_claim.bluffRate += 1

            if isinstance(adversaire, AdversaireIA):
                adversaire.denonceReussi += 1

    else:  # Si l'adversaire n'accuse pas de bluff
        adversaire.ajouter_points(1)  # Le bluffeur (ou le joueur honnête) gagne 1 point sans être contesté

        if isinstance(adversaire, AdversaireIA):
            adversaire.bluffReussis += 1

    if isinstance(ia, AdversaireIA):
      ia.recalcul_proba(joueur1,numeroManche)


def partie():
    numManche = 0
    deck = Deck()
    joueur1 = Joueur("Joueur 1")  # Probabilités définies ici
    ia = AdversaireIA("IA")  # Probabilités définies ici
    distribuer(deck, joueur1, ia)
    while deck.cartes:
        numManche += 1
        carte_retournee = deck.tirer_carte()
        manche(carte_retournee, joueur1, ia,numManche)
     #print(ia.cptAccuse/numManche)
    return joueur1.points, ia.points



In [2]:
def simulation(nb_parties=100):
    victoires_joueur1 = 0
    victoires_ia = 0
    for _ in range(nb_parties):
        points_joueur1, points_ia = partie()
        if points_joueur1 < points_ia:
            victoires_joueur1 += 1
        else:
            victoires_ia += 1
    print(f"Après {nb_parties} parties : Joueur 1 a gagné {victoires_joueur1} fois, IA a gagné {victoires_ia} fois.")

simulation()

Après 100 parties : Joueur 1 a gagné 3 fois, IA a gagné 97 fois.
